In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("tickerData") \
       .getOrCreate()

spark

In [ ]:
from statsmodels.tsa.stattools import coint
tickers = sorted(list(set([
    "AFFLE.NS",
    "APLAPOLLO.NS",
    "APARINDS.NS",
    "APOLLOTYRE.NS",
    "ARE&M.NS",
    "BANDHANBNK.BO",
    "BBTC.NS",
    "BECTORFOOD.NS",
    "CIPLA.NS",
    "COFORGE.NS",
    "DRREDDY.NS",
    "ELECON.NS",
    "ENDURANCE.NS",
    "ENGINERSIN.NS",
    "ETERNAL.NS",
    "FCL.BO",
    "FINCABLES.BO",
    "FINPIPE.NS",
    "GOLDBEES.NS",
    "GOLDIAM.NS",
    "HAPPSTMNDS.NS",
    "HINDUNILVR.BO",
    "HINDWAREAP.NS",
    "HSCL.BO",
    "HSCL.NS",
    "HDFCBANK.BO",
    "INDGELA.BO",
    "INDHOTEL.NS",
    "ITI.NS",
    "INFY.NS",
    "ITC.NS",
    "JSL.NS",
    "JSWINFRA.NS",
    "KIRLOSBROS.NS",
    "KIRLOSENG.NS",
    "KPITTECH.NS",
    "LUPIN.BO",
    "LTTS.NS",
    "NEWGEN.NS",
    "ONGC.NS",
    "PRAJASH.BO",
    "PRAJIND.NS",
    "PRICOLLTD.BO",
    "POLICYBZR.NS",
    "RICOAUTO.NS",
    "RCF.NS",
    "REFEX.NS",
    "RPPINFRA.NS",
    "ROSSARI.BO",
    "RVNL.NS",
    "SBIN.NS",
    "SANGHVIMOV.BO",
    "SETFNIF50.BO",
    "TANLA.NS",
    "TCS.BO",
    "TCS.NS",
    "WABAG.NS",
    "WAAREERTL.BO"
])))
for i in tickers:
  data = yf.download(i, period="2y", interval="1d")
  data.to_csv(f"{i}.csv")
  data_df= spark.read.csv(f"{i}.csv", header=True, inferSchema=True)
  data_df= data_df.withColumn("Ticker", lit(f"{i}"))
  # Filter out rows where 'Price' column contains "Ticker" or "Date"
  data_df_filtered = data_df.filter((col("Price") != lit("Ticker")) & (col("Price") != lit("Date")))\
  .withColumnRenamed("Price", "Date")
  # Write the filtered data_df to the CSV
  data_df_filtered.write.option("header","true")\
  .mode("append")\
  .csv("1_year_history.csv")

  # The commented out code below seems to be an alternative approach that is not needed with the current fix.
  # data_df = spark.createDataFrame(data)
  # data_df_renamed = data_df
  # for j in data_df.columns:
  #   data_df_renamed = data_df_renamed.withColumnRenamed(j, j.split(",")[0][2:-1])
  # data_df_ticker = data_df_renamed.withColumn("Ticker", lit(f"{i}"))
  # data_df_ticker.write.option("header","true")\
  # .mode("append")\
  # .csv("1_year_history.csv")

In [ ]:
ticker_final_df = spark.read.csv("1_year_history.csv", header=True, inferSchema=True)

In [ ]:
ticker_final_df.select("Ticker").distinct().show()

In [ ]:
one_yr_df = ticker_final_df.toPandas()
one_yr_df.head()

In [ ]:
# tickers = ["INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS"]
# Initialize an empty DataFrame to store the combined results
combined_tick_df = pd.DataFrame()

for i in tickers:
  tick_one_yr_df = ticker_final_df.select("*").filter(col("Ticker") == lit(f"{i}")).orderBy("Date").toPandas()

  # Convert relevant columns to numeric, coercing errors
  numeric_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
  for col_name in numeric_cols:
      tick_one_yr_df[col_name] = pd.to_numeric(tick_one_yr_df[col_name], errors='coerce')

  #calculating SMA
  tick_one_yr_df['SMA20'] = tick_one_yr_df['Close'].rolling(window=20).mean()
  tick_one_yr_df['SMA50'] = tick_one_yr_df['Close'].rolling(window=50).mean()
  tick_one_yr_df['SMA200'] = tick_one_yr_df['Close'].rolling(window=200).mean()

  #calculating MACD
  ema12 = tick_one_yr_df['Close'].ewm(span=12, adjust=False).mean()
  ema26 = tick_one_yr_df['Close'].ewm(span=26, adjust=False).mean()
  tick_one_yr_df['MACD'] = ema12 - ema26
  tick_one_yr_df['MACD_Signal'] = tick_one_yr_df['MACD'].ewm(span=9, adjust=False).mean()

  #calculating RSI
  delta = tick_one_yr_df['Close'].diff()
  gain = delta.clip(lower=0)
  loss = -delta.clip(upper=0)
  # Wilder's smoothing (EMA-like)
  avg_gain = gain.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
  avg_loss = loss.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
  rs = avg_gain / avg_loss
  tick_one_yr_df['RSI'] = 100 - (100 / (1 + rs))



  #calculating william %R
  high14 = tick_one_yr_df['High'].rolling(window=14).max()
  low14 = tick_one_yr_df['Low'].rolling(window=14).min()
  tick_one_yr_df['Williams_%R'] = -100 * (high14 - tick_one_yr_df['Close']) / (high14 - low14)

  #calculating MFI
  tp = (tick_one_yr_df['High'] + tick_one_yr_df['Low'] + tick_one_yr_df['Close']) / 3
  mf = tp * tick_one_yr_df['Volume']
  pos_mf = mf.where(tp > tp.shift(1), 0)
  neg_mf = mf.where(tp < tp.shift(1), 0)
  rolling_pos = pos_mf.rolling(window=14).sum()
  rolling_neg = neg_mf.rolling(window=14).sum()
  # Avoid division by zero
  mfr = rolling_pos / (rolling_neg.replace(0, np.nan))
  tick_one_yr_df['MFI'] = 100 - (100 / (1 + mfr))

  #calculating cci
  tp = (tick_one_yr_df['High'] + tick_one_yr_df['Low'] + tick_one_yr_df['Close']) / 3
  sma_tp = tp.rolling(window=20).mean()
  mad = tp.rolling(window=20).apply(lambda x: (x - x.mean()).abs().mean(), raw=False) # Removed raw=True
  tick_one_yr_df['cci'] = (tp - sma_tp) / (0.015 * mad)

  # Calculating On-Balance Volume (OBV)
  # Initialize OBV with 0
  tick_one_yr_df['OBV'] = 0
  # Calculate OBV based on price changes using .loc for assignment
  for j in range(1, len(tick_one_yr_df)):
      if tick_one_yr_df['Close'].iloc[j] > tick_one_yr_df['Close'].iloc[j-1]:
          tick_one_yr_df.loc[j, 'OBV'] = tick_one_yr_df['OBV'].iloc[j-1] + tick_one_yr_df['Volume'].iloc[j]
      elif tick_one_yr_df['Close'].iloc[j] < tick_one_yr_df['Close'].iloc[j-1]:
          tick_one_yr_df.loc[j, 'OBV'] = tick_one_yr_df['OBV'].iloc[j-1] - tick_one_yr_df['Volume'].iloc[j]
      else:
          tick_one_yr_df.loc[j, 'OBV'] = tick_one_yr_df['OBV'].iloc[j-1]

  # Calculating Accumulation/Distribution Line (A/D Line)
  # Money Flow Multiplier (MFM)
  # Ensure High and Low are not equal to avoid division by zero
  mfm_denominator = tick_one_yr_df['High'] - tick_one_yr_df['Low']
  mfm = ((tick_one_yr_df['Close'] - tick_one_yr_df['Low']) - (tick_one_yr_df['High'] - tick_one_yr_df['Close'])) / mfm_denominator
  mfm = mfm.replace([np.inf, -np.inf], np.nan).fillna(0) # Handle potential division by zero and NaNs
  # Money Flow Volume (MFV)
  mfv = mfm * tick_one_yr_df['Volume']
  # Accumulation/Distribution Line (A/D Line) is the cumulative sum of MFV
  tick_one_yr_df['AD_Line'] = mfv.cumsum()


  # Append the processed DataFrame for the current ticker to the combined DataFrame
  combined_tick_df = pd.concat([combined_tick_df, tick_one_yr_df], ignore_index=True)

# Now combined_tick_df contains the data for all tickers with SMA values and new indicators

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Convert 'Date' column to datetime objects if not already
combined_tick_df['Date'] = pd.to_datetime(combined_tick_df['Date'])

# Get the list of unique tickers
unique_tickers = combined_tick_df['Ticker'].unique()

# Plot for each ticker
for ticker in unique_tickers:
    ticker_df = combined_tick_df[combined_tick_df['Ticker'] == ticker].copy()

    plt.figure(figsize=(12, 6))
    plt.plot(ticker_df['Date'], ticker_df['Close'] , label='Close Price')
    plt.plot(ticker_df['Date'], ticker_df['SMA20'] , label='SMA 20')
    plt.plot(ticker_df['Date'], ticker_df['SMA50'] , label='SMA 50')
    plt.plot(ticker_df['Date'], ticker_df['SMA200'] , label='SMA 200')

    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'{ticker} Stock Price and SMAs over Time')
    plt.legend()
    plt.grid(True)

    # Format x-axis to show months
    ax = plt.gca()
    formatter = mdates.DateFormatter('%Y-%m')
    ax.xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# Convert 'Date' column to datetime objects if not already
combined_tick_df['Date'] = pd.to_datetime(combined_tick_df['Date'])

# Get the list of unique tickers
unique_tickers = combined_tick_df['Ticker'].unique()

# Plot for each ticker with subplots for each indicator
indicators = ['RSI', 'MFI', 'MACD', 'Williams_%R', 'cci', 'OBV', 'AD_Line'] # Added OBV and AD_Line

for ticker in unique_tickers:
    ticker_df = combined_tick_df[combined_tick_df['Ticker'] == ticker].copy()

    # Create a figure with subplots for each indicator
    fig, axes = plt.subplots(nrows=len(indicators), ncols=1, figsize=(12, 2 * len(indicators)), sharex=True)
    fig.suptitle(f'{ticker} Technical Indicators over Time', y=1.02) # Add a title for the entire figure

    for i, indicator in enumerate(indicators):
        ax = axes[i]
        ax.plot(ticker_df['Date'], ticker_df[indicator], label=indicator)

        # Add threshold lines and shaded areas based on the indicator
        if indicator == 'MACD':
            ax.axhline(0, color='red', linestyle='--', label='Zero Line')
            # Shade area above zero
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], 0, where=(ticker_df[indicator] > 0), color='green', alpha=0.3, interpolate=True)
            # Shade area below zero
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], 0, where=(ticker_df[indicator] < 0), color='red', alpha=0.3, interpolate=True)

            # Overlay MACD Signal
            ax.plot(ticker_df['Date'], ticker_df['MACD_Signal'], label='MACD Signal', linestyle='--')

            # Display current values and difference in the top right corner
            current_macd = ticker_df['MACD'].iloc[-1]
            current_signal = ticker_df['MACD_Signal'].iloc[-1]
            macd_diff = current_macd - current_signal
            text_to_display_corner = f'MACD: {current_macd:.2f}\nSignal: {current_signal:.2f}\nDiff: {macd_diff:.2f}'
            ax.text(1, 1, text_to_display_corner, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

            # Display current MACD value at the center
            ax.text(0.5, 0.5, f'Current MACD: {current_macd:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))


        elif indicator == 'Williams_%R':
            ax.axhline(-20, color='red', linestyle='--', label='Upper Threshold (-20)')
            ax.axhline(-80, color='red', linestyle='--', label='Lower Threshold (-80)')
            # Shade area above -20
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], -20, where=(ticker_df[indicator] > -20), color='red', alpha=0.3, interpolate=True)
            # Shade area below -80
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], -80, where=(ticker_df[indicator] < -80), color='green', alpha=0.3, interpolate=True)

            # Display current value at the center
            current_value = ticker_df[indicator].iloc[-1]
            ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))


        elif indicator == 'RSI':
            ax.axhline(70, color='red', linestyle='--', label='Upper Threshold (70)')
            ax.axhline(30, color='red', linestyle='--', label='Lower Threshold (30)')
            # Shade area above 70
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], 70, where=(ticker_df[indicator] > 70), color='red', alpha=0.3, interpolate=True)
            # Shade area below 30
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], 30, where=(ticker_df[indicator] < 30), color='green', alpha=0.3, interpolate=True)
            current_value = ticker_df[indicator].iloc[-1]
            ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))


        elif indicator == 'MFI':
            ax.axhline(80, color='red', linestyle='--', label='Upper Threshold (80)')
            ax.axhline(20, color='red', linestyle='--', label='Lower Threshold (20)')
            # Shade area above 80
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], 80, where=(ticker_df[indicator] > 80), color='red', alpha=0.3, interpolate=True)
            # Shade area below 20
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], 20, where=(ticker_df[indicator] < 20), color='green', alpha=0.3, interpolate=True)
            current_value = ticker_df[indicator].iloc[-1]
            ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

        elif indicator == 'cci': # Added CCI plotting logic
            ax.axhline(200, color='red', linestyle='--', label='Upper Threshold (200)')
            ax.axhline(-200, color='red', linestyle='--', label='Lower Threshold (-200)')
            # Shade area above 200
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], 200, where=(ticker_df[indicator] > 200), color='red', alpha=0.3, interpolate=True)
            # Shade area below -200
            ax.fill_between(ticker_df['Date'], ticker_df[indicator], -200, where=(ticker_df[indicator] < -200), color='green', alpha=0.3, interpolate=True)
            current_value = ticker_df[indicator].iloc[-1]
            ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

        elif indicator == 'OBV':
             # Display current value at the center
            current_value = ticker_df[indicator].iloc[-1]
            ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

        elif indicator == 'AD_Line':
             # Display current value at the center
            current_value = ticker_df[indicator].iloc[-1]
            ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))


        ax.set_ylabel(indicator)
        ax.legend()
        ax.grid(True)

        # Format x-axis to show months for the bottom subplot
        if i == len(indicators) - 1:
            formatter = mdates.DateFormatter('%Y-%m')
            ax.xaxis.set_major_formatter(formatter)
            plt.xticks(rotation=45)
        else:
            ax.tick_params(labelbottom=False) # Hide x-axis labels for upper subplots


    plt.tight_layout()
    plt.show()

In [ ]:
filtered_tickers_df = pd.DataFrame()

for ticker in combined_tick_df['Ticker'].unique():
    ticker_df = combined_tick_df[combined_tick_df['Ticker'] == ticker].copy()

    # Calculate previous day's values
    ticker_df['RSI_prev'] = ticker_df['RSI'].shift(1)
    ticker_df['MACD_prev'] = ticker_df['MACD'].shift(1)
    ticker_df['Williams_%R_prev'] = ticker_df['Williams_%R'].shift(1)
    ticker_df['cci_prev'] = ticker_df['cci'].shift(1)


    # Define the filtering conditions based on the latest proposal
    condition_rsi = (ticker_df['RSI'] >= 55) & (ticker_df['RSI'] > ticker_df['RSI_prev'])
    condition_williams = (ticker_df['Williams_%R'] >= -22) & (ticker_df['Williams_%R'] > ticker_df['Williams_%R_prev'])
    condition_macd = ((ticker_df['MACD'] - ticker_df['MACD_Signal']) > 0) & (ticker_df['MACD'] > ticker_df['MACD_prev'])
    # Keep CCI condition for potential future use or as an optional filter
    condition_cci = (ticker_df['cci'] > -200) & (ticker_df['cci'] < 200)

    # Combine all conditions
    all_conditions = condition_rsi & condition_macd & condition_williams & condition_cci # Added CCI to combined conditions


    # Filter the DataFrame based on all conditions
    filtered_df = ticker_df[all_conditions]

    # Append the filtered data for the current ticker to the result DataFrame
    filtered_tickers_df = pd.concat([filtered_tickers_df, filtered_df], ignore_index=True)

# Display the tickers and dates that meet the criteria
display(filtered_tickers_df[['Date', 'Ticker', 'RSI', 'MACD', 'MACD_Signal', 'Williams_%R', 'cci']]) # Added CCI to display

In [ ]:
latest_two_days_df = pd.DataFrame()

for ticker in combined_tick_df['Ticker'].unique():
    ticker_df = combined_tick_df[combined_tick_df['Ticker'] == ticker].copy()

    # Sort by date to ensure latest dates are at the end
    ticker_df = ticker_df.sort_values(by='Date', ascending=True)

    # Get the latest two dates
    latest_dates = ticker_df['Date'].tail(2).tolist()

    # Filter for the latest two dates
    latest_two_days_ticker_df = ticker_df[ticker_df['Date'].isin(latest_dates)]

    # Append to the combined DataFrame
    latest_two_days_df = pd.concat([latest_two_days_df, latest_two_days_ticker_df], ignore_index=True)

# Now apply the filtering logic and categorization to the latest two days data
filtered_latest_two_days_df = pd.DataFrame()

for ticker in latest_two_days_df['Ticker'].unique():
    ticker_df = latest_two_days_df[latest_two_days_df['Ticker'] == ticker].copy()

    # Calculate previous day's values (within the latest two days)
    ticker_df['RSI_prev'] = ticker_df['RSI'].shift(1)
    ticker_df['MACD_prev'] = ticker_df['MACD'].shift(1)
    ticker_df['Williams_%R_prev'] = ticker_df['Williams_%R'].shift(1)
    # Assuming CCI should also be included based on previous discussions
    ticker_df['cci_prev'] = ticker_df['cci'].shift(1)


    # Define the filtering conditions based on the latest proposal
    condition_rsi = (ticker_df['RSI'] >= 55) & (ticker_df['RSI'] > ticker_df['RSI_prev'])
    condition_williams = (ticker_df['Williams_%R'] >= -22) & (ticker_df['Williams_%R'] > ticker_df['Williams_%R_prev'])
    condition_macd = ((ticker_df['MACD'] - ticker_df['MACD_Signal']) > 0) & (ticker_df['MACD'] > ticker_df['MACD_prev'])
    # Keep CCI condition for potential future use or as an optional filter
    condition_cci = (ticker_df['cci'] > -200) & (ticker_df['cci'] < 200) # Example CCI condition within thresholds


    # Count how many of the *three proposed* conditions are satisfied for each row
    ticker_df['satisfied_conditions_count'] = condition_rsi.astype(int) + \
                                               condition_williams.astype(int) + \
                                               condition_macd.astype(int)


    # Categorize based on the count (assuming the user still wants these categories based on the *new* count)
    ticker_df['Recommendation'] = 'None'
    # Diamond Pick: All 3 proposed conditions satisfied
    ticker_df.loc[ticker_df['satisfied_conditions_count'] == 3, 'Recommendation'] = 'Diamond Pick'
    # Golden Pick: Any 2 of the 3 proposed conditions satisfied
    ticker_df.loc[ticker_df['satisfied_conditions_count'] == 2, 'Recommendation'] = 'Golden Pick'
    # Silver Pick: Any 1 of the 3 proposed conditions satisfied
    ticker_df.loc[ticker_df['satisfied_conditions_count'] == 1, 'Recommendation'] = 'Silver Pick'


    # Create a description of satisfied conditions
    conditions_met = []
    if condition_rsi.any(): # Check if there's at least one True in the series
         conditions_met.append('RSI >= 55 & Increasing')
    if condition_williams.any():
        conditions_met.append('Williams %R >= -22 & Increasing')
    if condition_macd.any():
        conditions_met.append('MACD > Signal & Increasing')
    if condition_cci.any(): # Include CCI if you want to describe it when met
         conditions_met.append('CCI within thresholds')

    # Join the conditions with a comma; handle case where no conditions are met
    ticker_df['Satisfied_Conditions_Description'] = ticker_df.apply(
        lambda row: ', '.join([cond for cond, met in zip(['RSI >= 55 & Increasing', 'Williams %R >= -22 & Increasing', 'MACD > Signal & Increasing', 'CCI within thresholds'],
                                                         [row['RSI'] >= 55 and row['RSI'] > row['RSI_prev'],
                                                          row['Williams_%R'] >= -22 and row['Williams_%R'] > row['Williams_%R_prev'],
                                                          (row['MACD'] - row['MACD_Signal']) > 0 and row['MACD'] > row['MACD_prev'],
                                                          row['cci'] > -200 and row['cci'] < 200]) if met]), axis=1
    )


    # Filter for rows with 'Diamond Pick', 'Golden Pick', or 'Silver Pick'
    filtered_df = ticker_df[(ticker_df['Recommendation'] == 'Diamond Pick') | (ticker_df['Recommendation'] == 'Golden Pick') | (ticker_df['Recommendation'] == 'Silver Pick')]


    # Append the filtered data for the current ticker to the result DataFrame
    filtered_latest_two_days_df = pd.concat([filtered_latest_two_days_df, filtered_df], ignore_index=True)

# Display the tickers, dates, and recommendations
display(filtered_latest_two_days_df[['Date', 'Ticker', 'RSI', 'MACD', 'MACD_Signal', 'Williams_%R', 'cci', 'satisfied_conditions_count', 'Recommendation', 'Satisfied_Conditions_Description']])

In [ ]:
combined_tick_df.count()

# Task
Send an email to "Mail_id@gmail.com" containing the shortlisted tickers from the `filtered_latest_two_days_df` DataFrame and their corresponding technical indicator plots, using the Gmail address and app password stored in Colab secrets.

## Access gmail credentials

### Subtask:
Securely retrieve the Gmail address and app password from Colab secrets.


**Reasoning**:
Access the Gmail address and app password from Colab secrets using the `google.colab` library.



In [ ]:
from google.colab import userdata

gmail_address = userdata.get('GMAIL_ADDRESS')
gmail_app_password = userdata.get('GMAIL_APP_PASSWORD')

print("Gmail address retrieved.")

## Access gmail credentials

### Subtask:
Retry securely retrieving the Gmail address and app password from Colab secrets after the previous attempt failed due to a `SecretNotFoundError`.


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Email details
sender_email = gmail_address
sender_password = gmail_app_password
receiver_emails = ["Mail_id@gmail.com", sender_email] # Include the sender's email as a recipient
subject = "Shortlisted Stock Tickers and Technical Analysis Plots" # Updated subject

# Prepare the email body with the shortlisted tickers data
if not filtered_latest_two_days_df.empty:
    email_body = "Dear Investor,\n\nHere are the shortlisted stock tickers based on your criteria for the latest two trading sessions:\n\n"
    # Convert shortlisted tickers DataFrame to HTML table
    email_body += filtered_latest_two_days_df[['Date', 'Ticker', 'Recommendation', 'satisfied_conditions_count', 'Satisfied_Conditions_Description']].to_html(index=False)
    email_body += "\n\nPlease find the technical analysis plots for these tickers attached.\n\n"

    # Removed the section that adds performance metrics to the email body


else:
    email_body = "Dear Investor,\n\nNo tickers met your criteria for the latest two trading sessions."

# Create the MIMEMultipart message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = ", ".join(receiver_emails)
msg['Subject'] = subject

# Create a MIMEText part with HTML content
html_part = MIMEText(email_body, 'html')
msg.attach(html_part)

# Generate and attach plots for shortlisted tickers
temp_plot_files = [] # List to keep track of temporary plot files

if not filtered_latest_two_days_df.empty:
    unique_shortlisted_tickers = filtered_latest_two_days_df['Ticker'].unique()
    for ticker in unique_shortlisted_tickers:
        # Find the latest date entry for the current ticker in the filtered DataFrame
        latest_date_entry = filtered_latest_two_days_df[filtered_latest_two_days_df['Ticker'] == ticker].iloc[-1]
        date_of_signal = latest_date_entry['Date'].strftime('%Y-%m-%d')

        # Find the corresponding full data for plotting from the combined_tick_df
        ticker_plot_df = combined_tick_df[combined_tick_df['Ticker'] == ticker].copy()

        # --- Generate a single figure with subplots for SMA and Technical Indicators ---
        indicators = ['RSI', 'MFI', 'MACD', 'Williams_%R', 'cci', 'OBV', 'AD_Line'] # Added OBV and AD_Line
        num_indicators = len(indicators)
        fig, axes = plt.subplots(nrows=num_indicators + 1, ncols=1, figsize=(12, 2 * (num_indicators + 1)), sharex=True)
        fig.suptitle(f'{ticker} Technical Analysis ({date_of_signal})', y=1.02) # Add a title for the entire figure

        # Plot SMA on the top subplot
        ax_sma = axes[0]
        ax_sma.plot(ticker_plot_df['Date'], ticker_plot_df['Close'] , label='Close Price') # Display actual price
        ax_sma.plot(ticker_plot_df['Date'], ticker_plot_df['SMA20'] , label='SMA 20')
        ax_sma.plot(ticker_plot_df['Date'], ticker_plot_df['SMA50'] , label='SMA 50')
        ax_sma.plot(ticker_plot_df['Date'], ticker_plot_df['SMA200'] , label='SMA 200')

        ax_sma.set_ylabel('Price') # Changed ylabel to Price
        ax_sma.set_title('Stock Price and SMAs')
        ax_sma.legend()
        ax_sma.grid(True)

        # Display current values of Close and SMAs at the bottom left
        current_close = ticker_plot_df['Close'].iloc[-1]
        current_sma20 = ticker_plot_df['SMA20'].iloc[-1]
        current_sma50 = ticker_plot_df['SMA50'].iloc[-1]
        current_sma200 = ticker_plot_df['SMA200'].iloc[-1]
        text_to_display_sma = f'Close: {current_close:.2f}\nSMA20: {current_sma20:.2f}\nSMA50: {current_sma50:.2f}\nSMA200: {current_sma200:.2f}'
        ax_sma.text(0.01, 0.01, text_to_display_sma, horizontalalignment='left', verticalalignment='bottom', transform=ax_sma.transAxes, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))


        # Plot Technical Indicators on the subsequent subplots
        for i, indicator in enumerate(indicators):
            ax = axes[i + 1] # Start from the second subplot
            ax.plot(ticker_plot_df['Date'], ticker_plot_df[indicator], label=indicator)

            # Add threshold lines and shaded areas based on the indicator (replicated from b0ed485d)
            if indicator == 'MACD':
                ax.axhline(0, color='red', linestyle='--', label='Zero Line')
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], 0, where=(ticker_plot_df[indicator] > 0), color='green', alpha=0.3, interpolate=True)
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], 0, where=(ticker_plot_df[indicator] < 0), color='red', alpha=0.3, interpolate=True)
                ax.plot(ticker_plot_df['Date'], ticker_plot_df['MACD_Signal'], label='MACD Signal', linestyle='--')
                current_macd = ticker_plot_df['MACD'].iloc[-1]
                current_signal = ticker_plot_df['MACD_Signal'].iloc[-1]
                macd_diff = current_macd - current_signal
                text_to_display_corner = f'MACD: {current_macd:.2f}\nSignal: {current_signal:.2f}\nDiff: {macd_diff:.2f}'
                ax.text(1, 1, text_to_display_corner, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))
                ax.text(0.5, 0.5, f'Current MACD: {current_macd:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

            elif indicator == 'Williams_%R':
                ax.axhline(-20, color='red', linestyle='--', label='Upper Threshold (-20)')
                ax.axhline(-80, color='red', linestyle='--', label='Lower Threshold (-80)')
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], -20, where=(ticker_plot_df[indicator] > -20), color='red', alpha=0.3, interpolate=True)
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], -80, where=(ticker_plot_df[indicator] < -80), color='green', alpha=0.3, interpolate=True)
                current_value = ticker_plot_df[indicator].iloc[-1]
                ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

            elif indicator == 'RSI':
                ax.axhline(70, color='red', linestyle='--', label='Upper Threshold (70)')
                ax.axhline(30, color='red', linestyle='--', label='Lower Threshold (30)')
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], 70, where=(ticker_plot_df[indicator] > 70), color='red', alpha=0.3, interpolate=True)
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], 30, where=(ticker_plot_df[indicator] < 30), color='green', alpha=0.3, interpolate=True)
                current_value = ticker_plot_df[indicator].iloc[-1]
                ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

            elif indicator == 'MFI':
                ax.axhline(80, color='red', linestyle='--', label='Upper Threshold (80)')
                ax.axhline(20, color='red', linestyle='--', label='Lower Threshold (20)')
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], 80, where=(ticker_plot_df[indicator] > 80), color='red', alpha=0.3, interpolate=True)
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], 20, where=(ticker_plot_df[indicator] < 20), color='green', alpha=0.3, interpolate=True)
                current_value = ticker_plot_df[indicator].iloc[-1]
                ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

            elif indicator == 'cci':
                ax.axhline(200, color='red', linestyle='--', label='Upper Threshold (200)')
                ax.axhline(-200, color='red', linestyle='--', label='Lower Threshold (-200)')
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], 200, where=(ticker_plot_df[indicator] > 200), color='red', alpha=0.3, interpolate=True)
                ax.fill_between(ticker_plot_df['Date'], ticker_plot_df[indicator], -200, where=(ticker_plot_df[indicator] < -200), color='green', alpha=0.3, interpolate=True)
                current_value = ticker_plot_df[indicator].iloc[-1]
                ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

            elif indicator == 'OBV':
                 # Display current value at the center
                current_value = ticker_plot_df[indicator].iloc[-1]
                ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

            elif indicator == 'AD_Line':
                 # Display current value at the center
                current_value = ticker_plot_df[indicator].iloc[-1]
                ax.text(0.5, 0.5, f'Current {indicator}: {current_value:.2f}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))


            ax.set_ylabel(indicator)
            ax.legend()
            ax.grid(True)

            # Format x-axis to show months for the bottom subplot
            if i == num_indicators - 1:
                formatter = mdates.DateFormatter('%Y-%m')
                ax.xaxis.set_major_formatter(formatter)
                plt.xticks(rotation=45)
            else:
                ax.tick_params(labelbottom=False) # Hide x-axis labels for upper subplots


        plt.tight_layout()

        # Save the combined plot to a temporary file
        combined_plot_filename = f'{ticker}_technical_analysis_plot.png'
        plt.savefig(combined_plot_filename)
        plt.close(fig) # Close the figure to free up memory
        temp_plot_files.append(combined_plot_filename) # Add to list for cleanup

        # Attach the combined plot to the email
        with open(combined_plot_filename, 'rb') as f:
            img = MIMEImage(f.read())
            img.add_header('Content-Disposition', 'attachment', filename=combined_plot_filename)
            msg.attach(img)


# Connect to the Gmail SMTP server and send the email
try:
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_emails, msg.as_string())
    print("Email sent successfully!")

except Exception as e:
    print(f"Error sending email: {e}")

finally:
    # Clean up temporary plot files
    for plot_file in temp_plot_files:
        if os.path.exists(plot_file):
            os.remove(plot_file)
    if temp_plot_files: # Only print if there were files to remove
        print("Temporary plot files removed.")

# Task
Backtest the historical performance of the "Diamond Pick," "Golden Pick," and "Silver Pick" recommendations based on the indicator filtering logic in cell `0e47eb98`.

## Define backtesting period

### Subtask:
Determine the historical date range over which you want to perform the backtesting.


**Reasoning**:
Determine the earliest date after which all technical indicators are calculated in `combined_tick_df` to set the backtesting start date, and set the end date to the latest date in the data.



In [ ]:
# Determine the earliest date after which all indicators have valid data
# The SMA200 has the longest lookback period (200 days)
# We need at least 200 data points for SMA200 to have a value
earliest_date_with_all_indicators = combined_tick_df.dropna(subset=['SMA200', 'RSI', 'MFI', 'MACD', 'Williams_%R', 'cci']).groupby('Ticker')['Date'].min().max()

# Determine the latest date in the data
latest_date = combined_tick_df['Date'].max()

print(f"Backtesting start date: {earliest_date_with_all_indicators.strftime('%Y-%m-%d')}")
print(f"Backtesting end date: {latest_date.strftime('%Y-%m-%d')}")

# Define the backtesting period
backtesting_dates = combined_tick_df[(combined_tick_df['Date'] >= earliest_date_with_all_indicators) & (combined_tick_df['Date'] <= latest_date)]['Date'].unique()
backtesting_dates = sorted(backtesting_dates)

In [ ]:
# Initialize a list to store historical picks
historical_picks = []

# Iterate through each date in the backtesting period
for current_date in backtesting_dates:
    # Filter data up to the current date for all tickers
    historical_data_up_to_date = combined_tick_df[combined_tick_df['Date'] <= current_date].copy()

    # Apply the filtering logic and categorization for the current date
    for ticker in historical_data_up_to_date['Ticker'].unique():
        ticker_df = historical_data_up_to_date[historical_data_up_to_date['Ticker'] == ticker].copy()

        # Ensure there are at least two days of data for this ticker up to the current date to calculate previous day values
        if ticker_df.shape[0] >= 2:
            # Get the data for the current date and the previous day
            latest_two_days_ticker_df = ticker_df.tail(2).copy()


            # Calculate previous day's values (within the latest two days)
            latest_two_days_ticker_df['RSI_prev'] = latest_two_days_ticker_df['RSI'].shift(1)
            latest_two_days_ticker_df['MACD_prev'] = latest_two_days_ticker_df['MACD'].shift(1)
            latest_two_days_ticker_df['Williams_%R_prev'] = latest_two_days_ticker_df['Williams_%R'].shift(1)
            latest_two_days_ticker_df['cci_prev'] = latest_two_days_ticker_df['cci'].shift(1)


            # Apply the filtering conditions for the latest day in this window
            current_day_data = latest_two_days_ticker_df.iloc[-1]
            previous_day_data = latest_two_days_ticker_df.iloc[-2]

            # Define the filtering conditions based on the logic in cell 0e47eb98
            condition_rsi = (current_day_data['RSI'] >= 55) and (current_day_data['RSI'] > previous_day_data['RSI'])
            condition_williams = (current_day_data['Williams_%R'] >= -22) and (current_day_data['Williams_%R'] > previous_day_data['Williams_%R'])
            condition_macd = ((current_day_data['MACD'] - current_day_data['MACD_Signal']) > 0) and (current_day_data['MACD'] > previous_day_data['MACD'])
            condition_cci = (current_day_data['cci'] > -200) and (current_day_data['cci'] < 200)


            # Count how many conditions are satisfied (based on the 3 proposed conditions + optional CCI)
            # Note: The categorization in 0e47eb98 was based on 3 proposed conditions + optional CCI.
            # Let's align the backtesting to count the same set of conditions.
            satisfied_conditions_count = int(condition_rsi) + \
                                       int(condition_williams) + \
                                       int(condition_macd) + \
                                       int(condition_cci)


            # Determine the recommendation based on the count logic in cell 0e47eb98
            recommendation = 'None'
            if satisfied_conditions_count >= 3: # Diamond Pick: 3 or more satisfied conditions (including CCI)
                recommendation = 'Diamond Pick'
            elif satisfied_conditions_count == 2: # Golden Pick: Exactly 2 satisfied conditions (including CCI)
                recommendation = 'Golden Pick'
            elif satisfied_conditions_count == 1: # Silver Pick: Exactly 1 satisfied condition (including CCI)
                recommendation = 'Silver Pick'


            # Record the pick if it's not 'None'
            if recommendation != 'None':
                historical_picks.append({
                    'Date': current_date,
                    'Ticker': ticker,
                    'Recommendation': recommendation,
                    'satisfied_conditions_count': satisfied_conditions_count,
                    'Close_Price_on_Pick_Date': current_day_data['Close']
                })

# Convert the list of historical picks to a DataFrame
historical_picks_df = pd.DataFrame(historical_picks)

# Display the historical picks
display(historical_picks_df.head())
display(historical_picks_df.shape)

# Task
Visualize the cumulative returns of each recommendation type based on the historical picks.

## Calculate returns for historical picks

### Subtask:
For each historical pick in `historical_picks_df`, determine the price change or return over a defined period following the pick date.


**Reasoning**:
Calculate the returns for each historical pick over a defined holding period by finding the closing price on the pick date and the price after the holding period.



In [ ]:
# 1. Define the holding period
holding_period = 5 # Example: 5 trading days (approximately one week)

# Initialize columns for return and target date
historical_picks_df['Return'] = np.nan
historical_picks_df['Max_Price_in_Holding_Period'] = np.nan # New column for max price
historical_picks_df['Date_of_Max_Price'] = pd.NaT # New column for date of max price


# 2. Iterate through each historical pick to calculate returns
for index, row in historical_picks_df.iterrows():
    ticker = row['Ticker']
    pick_date = row['Date']
    price_on_pick_date = row['Close_Price_on_Pick_Date']

    # Calculate the end date for the holding period
    end_date_holding_period = pick_date + pd.Timedelta(days=holding_period)

    # Find the data for the specific ticker within the holding period (starting the day after the pick)
    holding_period_data = combined_tick_df[(combined_tick_df['Ticker'] == ticker) &
                                           (combined_tick_df['Date'] > pick_date) &
                                           (combined_tick_df['Date'] <= end_date_holding_period)].copy()

    # Ensure there is data in the holding period
    if not holding_period_data.empty:
        # Find the maximum 'High' price within the holding period
        max_price = holding_period_data['High'].max()
        # Find the date of the maximum price
        date_of_max_price = holding_period_data.loc[holding_period_data['High'].idxmax(), 'Date']


        # Calculate the return based on the maximum price, and multiply by 100 for percentage
        calculated_return = ((max_price - price_on_pick_date) / price_on_pick_date) * 100

        # Store the calculated return, max price, and date of max price
        historical_picks_df.loc[index, 'Return'] = calculated_return
        historical_picks_df.loc[index, 'Max_Price_in_Holding_Period'] = max_price
        historical_picks_df.loc[index, 'Date_of_Max_Price'] = date_of_max_price


# Display the updated DataFrame with returns and target dates
display(historical_picks_df.head())

In [ ]:
# Analyze performance metrics for each recommendation type

performance_metrics_by_recommendation = {}

for recommendation_type in historical_picks_df['Recommendation'].unique():
    category_picks = historical_picks_df[historical_picks_df['Recommendation'] == recommendation_type].copy()

    if not category_picks.empty:
        total_picks = len(category_picks)
        winning_picks = category_picks[category_picks['Return'] > 0]
        losing_picks = category_picks[category_picks['Return'] <= 0] # Consider 0 return as not a win

        win_rate = len(winning_picks) / total_picks if total_picks > 0 else 0
        loss_rate = len(losing_picks) / total_picks if total_picks > 0 else 0

        average_winning_return = winning_picks['Return'].mean() if not winning_picks.empty else 0
        average_losing_return = losing_picks['Return'].mean() if not losing_picks.empty else 0
        average_total_return = category_picks['Return'].mean()


        performance_metrics_by_recommendation[recommendation_type] = {
            'Total Picks': total_picks,
            'Winning Picks': len(winning_picks),
            'Losing Picks': len(losing_picks),
            'Win Rate (%)': win_rate * 100,
            'Loss Rate (%)': loss_rate * 100,
            'Average Winning Return (%)': average_winning_return, # Removed * 100
            'Average Losing Return (%)': average_losing_return, # Removed * 100
            'Average Total Return (%)': average_total_return # Removed * 100
        }

# Convert the performance metrics dictionary to a DataFrame for better display
performance_metrics_by_recommendation_df = pd.DataFrame.from_dict(performance_metrics_by_recommendation, orient='index')

# Display the performance metrics by recommendation type
print("Performance Metrics by Recommendation Type:")
display(performance_metrics_by_recommendation_df)


# Analyze performance metrics for each ticker
performance_metrics_by_ticker = {}

for ticker in historical_picks_df['Ticker'].unique():
    ticker_picks = historical_picks_df[historical_picks_df['Ticker'] == ticker].copy()

    if not ticker_picks.empty:
        total_picks = len(ticker_picks)
        winning_picks = ticker_picks[ticker_picks['Return'] > 0]
        losing_picks = ticker_picks[ticker_picks['Return'] <= 0] # Consider 0 return as not a win

        win_rate = len(winning_picks) / total_picks if total_picks > 0 else 0
        loss_rate = len(losing_picks) / total_picks if total_picks > 0 else 0

        average_winning_return = winning_picks['Return'].mean() if not winning_picks.empty else 0
        average_losing_return = losing_picks['Return'].mean() if not losing_picks.empty else 0
        average_total_return = ticker_picks['Return'].mean()


        performance_metrics_by_ticker[ticker] = {
            'Total Picks': total_picks,
            'Winning Picks': len(winning_picks),
            'Losing Picks': len(losing_picks),
            'Win Rate (%)': win_rate * 100,
            'Loss Rate (%)': loss_rate * 100,
            'Average Winning Return (%)': average_winning_return, # Removed * 100
            'Average Losing Return (%)': average_losing_return, # Removed * 100
            'Average Total Return (%)': average_total_return # Removed * 100
        }

# Convert the performance metrics dictionary to a DataFrame for better display
performance_metrics_by_ticker_df = pd.DataFrame.from_dict(performance_metrics_by_ticker, orient='index')

# Display the performance metrics by ticker
print("\nPerformance Metrics by Ticker:")
display(performance_metrics_by_ticker_df)


# Analyze performance metrics grouped by Ticker and Recommendation Type
performance_metrics_by_ticker_recommendation = {}

# Group by Ticker and Recommendation Type
grouped_performance = historical_picks_df.groupby(['Ticker', 'Recommendation'])

for name, group in grouped_performance:
    ticker = name[0]
    recommendation_type = name[1]
    category_picks = group.copy() # Use the group as the category_picks

    if not category_picks.empty:
        total_picks = len(category_picks)
        winning_picks = category_picks[category_picks['Return'] > 0]
        losing_picks = category_picks[category_picks['Return'] <= 0] # Consider 0 return as not a win

        win_rate = len(winning_picks) / total_picks if total_picks > 0 else 0
        loss_rate = len(losing_picks) / total_picks if total_picks > 0 else 0

        average_winning_return = winning_picks['Return'].mean() if not winning_picks.empty else 0
        average_losing_return = losing_picks['Return'].mean() if not losing_picks.empty else 0
        average_total_return = category_picks['Return'].mean()

        # Store metrics using a tuple (ticker, recommendation_type) as the key
        performance_metrics_by_ticker_recommendation[(ticker, recommendation_type)] = {
            'Total Picks': total_picks,
            'Winning Picks': len(winning_picks),
            'Losing Picks': len(losing_picks),
            'Win Rate (%)': win_rate * 100,
            'Loss Rate (%)': loss_rate * 100,
            'Average Winning Return (%)': average_winning_return, # Removed * 100
            'Average Losing Return (%)': average_losing_return, # Removed * 100
            'Average Total Return (%)': average_total_return # Removed * 100
        }

# Convert the performance metrics dictionary to a DataFrame for better display
performance_metrics_by_ticker_recommendation_df = pd.DataFrame.from_dict(performance_metrics_by_ticker_recommendation, orient='index')

# Rename the index for clarity
performance_metrics_by_ticker_recommendation_df.index.names = ['Ticker', 'Recommendation']


# Display the performance metrics grouped by Ticker and Recommendation Type
print("\nPerformance Metrics by Ticker and Recommendation Type:")
display(performance_metrics_by_ticker_recommendation_df)

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

# Email details
sender_email = gmail_address
sender_password = gmail_app_password
receiver_email = "Mail_id@gmail.com"
subject = "Performance Metrics for Shortlisted Tickers"

# Filter performance metrics for shortlisted tickers
if not filtered_latest_two_days_df.empty and 'performance_metrics_by_ticker_recommendation_df' in locals():
    shortlisted_tickers_list = filtered_latest_two_days_df['Ticker'].unique().tolist()
    # Filter the performance metrics DataFrame for only the shortlisted tickers
    performance_metrics_shortlisted = performance_metrics_by_ticker_recommendation_df.loc[shortlisted_tickers_list, :]

    # Prepare the email body with the performance metrics as an HTML table
    email_body = "Dear User,\n\nHere are the performance metrics for the shortlisted tickers (Grouped by Ticker and Recommendation):\n\n"
    email_body += performance_metrics_shortlisted.to_html() # Convert DataFrame to HTML table
    email_body += "\n\n"

    # Create the MIMEText message with HTML content
    msg = MIMEText(email_body, 'html') # Set content type to html
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Connect to the Gmail SMTP server and send the email
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email with performance metrics sent successfully!")

    except Exception as e:
        print(f"Error sending email: {e}")
else:
    print("No shortlisted tickers found or performance metrics DataFrame not available.")

## Calculate daily returns for each category

### Subtask:
Aggregate the returns of the picks for each day and each recommendation type to get daily portfolio returns for each category.


**Reasoning**:
Aggregate the returns by date and recommendation type and pivot the data for cumulative return calculation.



In [ ]:
# # Group by Date and Recommendation and calculate the mean return
# daily_returns = historical_picks_df.groupby(['Date', 'Recommendation'])['Return'].mean().reset_index()

# # Pivot the DataFrame to have Dates as index and Recommendation types as columns
# daily_portfolio_returns = daily_returns.pivot(index='Date', columns='Recommendation', values='Return')

# # Fill missing values with 0
# daily_portfolio_returns = daily_portfolio_returns.fillna(0)

# # Sort the DataFrame by date
# daily_portfolio_returns = daily_portfolio_returns.sort_index()

# # Display the daily portfolio returns
# display(daily_portfolio_returns.head())
# display(daily_portfolio_returns.shape)

## Calculate cumulative returns

### Subtask:
Calculate the cumulative returns for each recommendation type over the backtesting period.


**Reasoning**:
Calculate the cumulative returns for each recommendation category by adding 1 to the daily return and then calculating the cumulative product.



In [ ]:
# # Calculate cumulative returns
# cumulative_returns = (daily_portfolio_returns + 1).cumprod()

# # Display the head of the cumulative returns DataFrame
# display(cumulative_returns.head())

## Visualize cumulative returns

### Subtask:
Visualize the cumulative returns of each recommendation type based on the historical picks.


**Reasoning**:
Create a line plot of the cumulative returns of each recommendation type.



In [ ]:
# plt.figure(figsize=(12, 6))
# plt.plot(cumulative_returns.index, cumulative_returns['Diamond Pick'], label='Diamond Pick')
# plt.plot(cumulative_returns.index, cumulative_returns['Golden Pick'], label='Golden Pick')
# plt.plot(cumulative_returns.index, cumulative_returns['Silver Pick'], label='Silver Pick')
# plt.title('Cumulative Returns of Recommendation Types')
# plt.xlabel('Date')
# plt.ylabel('Cumulative Return')
# plt.legend()
# plt.grid(True)
# plt.show()

## Summary:

### Data Analysis Key Findings

* The analysis calculated the cumulative returns for three recommendation types: 'Diamond Pick', 'Golden Pick', and 'Silver Pick', based on historical picks over a 5-day holding period.
* The `daily_portfolio_returns` DataFrame, used for calculating cumulative returns, contained 297 days of data.
* A line plot was generated showing the cumulative performance of each recommendation type over time, titled 'Cumulative Returns of Recommendation Types'.

### Insights or Next Steps

* Analyze the generated plot to compare the performance of the different recommendation types and identify which category has historically performed best.
* Consider extending the analysis to include other holding periods or risk metrics to get a more comprehensive understanding of the recommendation types' performance.


# Task
Perform a 2-year historical backtesting analysis for each ticker based on three defined trading signals (`signal_1`, `signal_2`, and `signal_3`) and report the winning percentage for each signal.

## Define backtesting period

### Subtask:
Set the historical date range for the 2-year analysis.


**Reasoning**:
Determine the earliest and latest dates for the backtesting period based on the availability of technical indicators in `combined_tick_df`.

